In [ ]:
import math
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
from pathlib import Path


pickle_path = 'xxx/python/outputs/agent_advisor_xgboost_python'

with open(Path(pickle_path) / "agent_advisor_xgboost_python.pickle", "rb") as pickle_model:
    model = pd.read_pickle(pickle_model)


In [9]:
def score(sf_face_2_face, sf_call_inbound, sf_email_campaigns, advisor_hh_children, annuity_mkt_opp, advisor_advising_years, advisor_aum, advisor_annuity_selling_years, advisor_age, advisor_net_worth, advisor_credit_hist_mos, advisor_firm_changes, advisor_credit_score, wholesaler, region_ca, region_ny, sf_email_responses):
    "Output: EM_CLASSIFICATION, EM_EVENTPROBABILITY"

    try:
        global model
    except NameError:
        with open(Path(pickle_path) / "agent_advisor_xgboost_python.pickle", "rb") as pickle_model:
            model = pd.read_pickle(pickle_model)


    index=None
    if not isinstance(sf_face_2_face, pd.Series):
        index=[0]
    input_array = pd.DataFrame(
        {"sf_face_2_face": sf_face_2_face, "sf_call_inbound": sf_call_inbound,
        "sf_email_campaigns": sf_email_campaigns, "advisor_hh_children":
        advisor_hh_children, "annuity_mkt_opp": annuity_mkt_opp,
        "advisor_advising_years": advisor_advising_years, "advisor_aum": advisor_aum,
        "advisor_annuity_selling_years": advisor_annuity_selling_years, "advisor_age":
        advisor_age, "advisor_net_worth": advisor_net_worth, "advisor_credit_hist_mos":
        advisor_credit_hist_mos, "advisor_firm_changes": advisor_firm_changes,
        "advisor_credit_score": advisor_credit_score, "wholesaler": wholesaler,
        "region_ca": region_ca, "region_ny": region_ny, "sf_email_responses":
        sf_email_responses}, index=index
    )
    input_array = xgb.DMatrix(input_array)
    prediction = model.predict(input_array).tolist()
    print (prediction)

    # Check for numpy values and convert to a CAS readable representation
    if isinstance(prediction, np.ndarray):
        prediction = prediction.tolist()

    if input_array.num_row() == 1:
        if prediction[0] > 0.5:
            EM_CLASSIFICATION = "1"
        else:
            EM_CLASSIFICATION = "0"
        return EM_CLASSIFICATION, prediction[0]
    else:
        df = pd.DataFrame(prediction)
        proba = df[1]
        classifications = np.where(df[1] > 0.5, '1', '0')
        return pd.DataFrame({'EM_CLASSIFICATION': classifications, 'EM_EVENTPROBABILITY': proba})

In [10]:
results = score(1,0,2,3,0.8522,.996,-.51003,0.125,1.14,2.03,1.15,3,-.899,9,0,1,3)

[0.004211342427879572]
